In [108]:
# importing Selenium Libs
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException, NoSuchElementException

# this line find the Browser Driver and opened them
driver = Chrome(service=Service(ChromeDriverManager().install()))

# set window size
width = 1450
height = 950
driver.set_window_size(width, height)

driver.get('https://twitter.com')


In [109]:
# Importent functions
import time

def click_button(by:By, value: str):
    button = driver.find_element(by=by, value=value)
    button.click()
    time.sleep(1) 
    
    # this function for geting inputs webpages value
def add_input(by:By, value: str, text: str):
    field = driver.find_element(by=by, value=value)
    field.send_keys(text)
    time.sleep(1)
    
def enter(by:By, value:str):
    enter = driver.find_element(by=by, value=value)
    enter.send_keys(Keys.ENTER)


In [ ]:
# accept cookis of the project
click_button(by=By.XPATH,
    value='//div[@role="button" and contains(.//text(), "Accept all cookies")]')

In [111]:
# go to login section
click_button(by=By.XPATH, 
    value="//a[@data-testid='loginButton']")

In [112]:
# add email
add_input(by=By.XPATH, 
    value="//input[@autocomplete='username']", text='')

time.sleep(2)

click_button(by=By.XPATH, 
    value='//div[@role="button" and contains(.//text(), "Next")]')
    

In [113]:
# adding password for login and login
add_input(by=By.XPATH,
    value="//input[@autocomplete='current-password']", text='')

time.sleep(2)

click_button(by=By.XPATH,
    value="//div[@data-testid='LoginForm_Login_Button']")
        
    

In [114]:
# search function username
def search(username):
    add_input(by=By.XPATH, value="//input[@placeholder='Search']", text=username)
    enter(by=By.XPATH, value="//input[@placeholder='Search']")

In [119]:
# set Target ID
def user_account(username):
    user = driver.get('https://twitter.com/'+ username)

def user_account_media(username):
    driver.get('https://twitter.com/'+ username + '/media')

def user_account_likes(username):
    driver.get('https://twitter.com/'+ username + '/likes')

def user_account_profile(username):
    driver.get('https://twitter.com/'+ username + '/photo')



In [123]:
# Mining Data
# - first change to the user

import re


def user_profile_info(username):
    time.sleep(2)
    following_element = driver.find_element(by=By.XPATH, value=f"//a[@href='/{username}/following']")
    followers_element = driver.find_element(by=By.XPATH, value=f"//a[@href='/{username}/verified_followers']")
    
    following_text = following_element.text
    followers_text = followers_element.text
    
    # Define a regular expression pattern to match numbers with optional commas, decimal parts, and suffixes like "k" and "M"
    number_pattern = r'(\d{1,3}(,\d{3})*|\d+)(\.\d+)?[kK]?M?'
    
    # Search for the number pattern in the text and extract the matched number as a string
    following_match = re.search(number_pattern, following_text)
    followers_match = re.search(number_pattern, followers_text)
    
    # Convert the matched numbers to integers
    following = parse_number(following_match.group()) if following_match else 0
    followers = parse_number(followers_match.group()) if followers_match else 0

    # geting user Bio
    Bio = driver.find_element(by=By.XPATH, value="//div[@data-testid='UserDescription']").text
    Location = driver.find_element(by=By.XPATH, value="//span[@data-testid='UserLocation']").text
    Joine_date = driver.find_element(by=By.XPATH, value="//span[@data-testid='UserJoinDate']").text
    user_post = driver.find_element(By.XPATH, "//div[contains(@class, 'css-1rynq56') and contains(@class, 'r-dnmrzs') and contains(@class, 'r-1udh08x') and contains(@class, 'r-3s2u2q') and contains(@class, 'r-bcqeeo') and contains(@class, 'r-qvutc0') and contains(@class, 'r-37j5jr') and contains(@class, 'r-n6v787') and contains(@class, 'r-1cwl3u0') and contains(@class, 'r-16dba41')]").text

    return following, followers, Bio, Location, Joine_date, user_post

def user_profile_media(username):
    user_media = driver.find_element(By.XPATH, "//div[contains(text(), 'photos & videos')]").text

    return user_media

def user_profile_likes(username):
    user_likes = driver.find_element(By.XPATH, "//div[contains(text(), 'Likes')]").text

    return user_likes

def user_profile_image(username):
    image_element = driver.find_element(By.XPATH, '//div[@class="css-175oi2r r-1mlwlqe r-1udh08x r-417010"]/img')

    # Get the value of the src attribute
    user_profile = image_element.get_attribute('src')


    return user_profile

def parse_number(number_str):
    # Convert string representation of number to actual number
    if number_str.endswith('M'):
        return int(float(number_str[:-1]) * 1_000_000)
    elif number_str.endswith('k') or number_str.endswith('K'):
        return int(float(number_str[:-1]) * 1_000)
    else:
        return int(float(number_str.replace(',', '')))

In [126]:
from flask import Flask, jsonify, request
import datetime
import json
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/process', methods=['POST'])
def process_data():
    req_data = request.get_json()
    
    # Extracting relevant data from input JSON
    username = req_data.get("username")
    out_path = req_data.get("outPath")

    # time
    created_at = datetime.datetime.now()

    user_account(username)
    time.sleep(4)
    # Retrieve following and followers counts
    following, followers, Bio, Location, Joine_date, user_post  = user_profile_info(username)
    
    time.sleep(1)
    user_account_media(username)
    time.sleep(5)
    user_media = user_profile_media(username)

    time.sleep(1)
    user_account_likes(username)
    time.sleep(5)
    user_likes = user_profile_likes(username)

    time.sleep(1)
    user_account_profile(username)
    time.sleep(5)
    user_profile = user_profile_image(username)

    # Dummy output data (replace this with your actual data processing logic)
    output_data = {
        "username": username,
        "following": following,
        "followers": followers,
        "user_bio" : Bio,
        "user_location" : Location,
        "user_joine_dare" : Joine_date,
        "user_posts" : user_post,
        "user_media" : user_media,
        "user_likes" : user_likes,
        "user_profile": user_profile,
        "created_at" : str(created_at),
    }
    
    # Saving the output data to the specified outPath
    if out_path:
        with open(out_path, "w") as file:
            file.write(json.dumps(output_data))
    
    return jsonify(output_data), 200

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [07/Mar/2024 18:40:01] "POST /process HTTP/1.1" 200 -
